In [23]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math

In [2]:
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time

In [3]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [163]:
# Get season stats for 16-17 season to get starting elo for training data
nba_16_17_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2016-17&SeasonType=Regular%20Season"

In [164]:
# visit website
browser.visit(nba_16_17_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [165]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [166]:
df_init = df.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
game_dates = df_init['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]

In [167]:
date_index=0
newElo = {}
eloScores = []
for row in df_init.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
    newElo[row.Team] = eloUpdate
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [169]:
db.nba_team_elo_training.insert_one(elo_list)

In [4]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_trad_boxscores_url = "https://stats.nba.com/teams/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2017-18&SeasonType=Regular%20Season"

In [8]:
# create splinter browser
def init_browser(noShowBrowser):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=noShowBrowser)
    return browser

In [7]:
browser = init_browser(False)

In [9]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df_adv = df[0].drop(columns=['Season'])
# Rename column names
df_adv.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df_adv['GameDate'] = df_adv['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [10]:
# visit website
browser.visit(nba_17_18_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_ff = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [19]:
# visit website
browser.visit(nba_17_18_trad_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_trad = df.drop(['MIN', 'MatchUp', 'W/L'], axis=1)

In [21]:
result = pd.merge(df_adv, df_ff, on=['Team', 'GameDate'])
result.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%'],
      dtype='object')

In [22]:
result_final = pd.merge(result, df_trad, on=['Team', 'GameDate'])
result_final.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'PF', '+/-'],
      dtype='object')

In [28]:
teams = result_final['Team'].unique()

In [29]:
myteam = [team for team in teams]

In [30]:
myteam.sort()

In [94]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [37]:
db.nba_team_elo_training.insert_one(elo_list)

In [62]:
result_sorted = result_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
result_sorted['HomeGame'] = result_sorted['MatchUp'].apply(lambda x: 1 if x.split(' ')[1] == 'vs.' else 0)

In [64]:
def eloProbability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
def EloRating(homeTeam, roadTeam, K, homeTeamWins):
    # To calculate the Winning 
    # Probability of Player B 
    Pb = eloProbability(homeTeam, roadTeam) 

    # To calculate the Winning 
    # Probability of Player A 
    Pa = eloProbability(roadTeam, homeTeam) 

    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (homeTeamWins == True) : 
        homeTeam = homeTeam + K * (1 - Pa) 
        roadTeam = roadTeam + K * (0 - Pb) 


    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    else : 
        homeTeam = homeTeam + K * (0 - Pa)
        roadTeam = roadTeam + K * (1 - Pb)
    return homeTeam, roadTeam

In [66]:
team1 = 1000
team2 = 1000

In [67]:
team1, team2 = EloRating(team1, team2, 20, False)

In [69]:
team2

1010.0

In [170]:
game_dates = result_sorted['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]
game_dates[0]

numpy.datetime64('2017-10-17T00:00:00.000000000')

In [81]:
game_dates[1] == np.datetime64(result_sorted.iloc[0]['GameDate'])

False

In [76]:
result_sorted.iloc[0]['GameDate']

Timestamp('2017-10-17 00:00:00')

In [171]:
temp_df = result_sorted.copy()

In [172]:
temp_df.head()

,Team,MatchUp,GameDate,W/L,MIN,OffRtg,DefRtg,NetRtg,AST%,AST/TO,...,OREB,DREB,REB,AST,TOV,STL,BLK,PF,+/-,HomeGame
0,GSW,GSW vs. HOU,2017-10-17,L,48,118.6,119.6,-1.0,79.1,2.00,...,6,35,41,34,17,5,9,25,-1,1
1,CLE,CLE vs. BOS,2017-10-17,W,48,104.1,100.0,4.1,50.0,1.12,...,9,41,50,19,17,3,4,25,3,1
2,HOU,HOU @ GSW,2017-10-17,W,48,119.6,118.6,1.0,59.6,2.15,...,10,33,43,28,13,9,5,16,1,0
3,BOS,BOS @ CLE,2017-10-17,L,48,100.0,104.1,-4.1,66.7,2.00,...,9,37,46,24,12,11,4,24,-3,0
4,IND,IND vs. BKN,2017-10-18,W,48,119.7,113.9,5.7,54.7,2.07,...,14,33,47,29,14,12,9,25,9,1


In [155]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [173]:
date_index=0
newElo = {}
eloScores = []
for row in temp_df.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
    newElo[row.Team] = eloUpdate
    eloScores.append(eloUpdate)
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [184]:
elo_list

{'ATL': 1375.7075741834688,
 'BKN': 1385.7473632514852,
 'BOS': 1593.1622149515758,
 'CHA': 1447.7280767309485,
 'CHI': 1367.1005274309066,
 'CLE': 1570.4270859757785,
 'DAL': 1343.0240681009527,
 'DEN': 1571.0319534263213,
 'DET': 1461.7873401532459,
 'GSW': 1619.4570676450153,
 'HOU': 1727.4508941343163,
 'IND': 1581.231834171123,
 'LAC': 1524.2948631384638,
 'LAL': 1451.6639776614604,
 'MEM': 1312.794183162584,
 'MIA': 1518.578999882534,
 'MIL': 1518.6617359287,
 'MIN': 1533.1207602745035,
 'NOP': 1588.8232069597443,
 'NYK': 1356.0233382357949,
 'OKC': 1586.408713208459,
 'ORL': 1336.8270182401138,
 'PHI': 1624.0357372854346,
 'PHX': 1286.05099854899,
 'POR': 1598.9654622637888,
 'SAC': 1390.6202815200002,
 'SAS': 1558.4988631578274,
 'TOR': 1652.5998449378783,
 'UTA': 1626.4434841314753,
 'WAS': 1491.7325313071099,
 '_id': ObjectId('5c5968398a95a602e4636635')}

In [175]:
len(eloScores)

2460

In [176]:
len(temp_df)

2460

In [177]:
temp_df['ELO'] = eloScores

In [178]:
temp_df.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'PF', '+/-', 'HomeGame', 'ELO'],
      dtype='object')

In [180]:
db.training_data.drop()

In [182]:
# export to json
items = temp_df.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [183]:
db.training_data.insert_many(items_db)

In [185]:
db.nba_team_elo_testing.insert_one(elo_list)

In [186]:
temp = db.nba_team_elo_testing.find()

In [188]:
temp = list(temp)

In [192]:
for i in temp:
    i.pop('_id', None)

In [194]:
elo_testing = temp[0]

In [195]:
elo_testing

{'ATL': 1375.7075741834688,
 'BKN': 1385.7473632514852,
 'BOS': 1593.1622149515758,
 'CHA': 1447.7280767309485,
 'CHI': 1367.1005274309066,
 'CLE': 1570.4270859757785,
 'DAL': 1343.0240681009527,
 'DEN': 1571.0319534263213,
 'DET': 1461.7873401532459,
 'GSW': 1619.4570676450153,
 'HOU': 1727.4508941343163,
 'IND': 1581.231834171123,
 'LAC': 1524.2948631384638,
 'LAL': 1451.6639776614604,
 'MEM': 1312.794183162584,
 'MIA': 1518.578999882534,
 'MIL': 1518.6617359287,
 'MIN': 1533.1207602745035,
 'NOP': 1588.8232069597443,
 'NYK': 1356.0233382357949,
 'OKC': 1586.408713208459,
 'ORL': 1336.8270182401138,
 'PHI': 1624.0357372854346,
 'PHX': 1286.05099854899,
 'POR': 1598.9654622637888,
 'SAC': 1390.6202815200002,
 'SAS': 1558.4988631578274,
 'TOR': 1652.5998449378783,
 'UTA': 1626.4434841314753,
 'WAS': 1491.7325313071099}

In [196]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [206]:
X_train = temp_df.loc[:,['TS%', 'TOV%', 'OREB%', 'FTARate', 'DefRtg', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%', 'ELO']]
y = temp_df['PTS']

In [207]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

In [208]:
scorePredictor = LinearRegression()

In [209]:
scorePredictor.fit(X_train_scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [210]:
scorePredictor.score(X_train_scaled, y)

0.781045061630136

In [211]:
scorePredictor.coef_

array([10.1279262 , -4.08551689,  3.26405907,  0.56223314, -1.23909495,
        1.04883183, -0.35987596, -0.16382807,  1.29840268, -0.21903964])